# Data Wrangling 2

Welcome to part 2! In this session, we will recap everything we covered in [Part 1](https://jackedtaylor.github.io/expra-wise24/introduction/data_wrangling_1.html), and we will cover:

* [Pipes](#pipes): `|>`
* [Reformatting into Wide / Long format](#wide-and-long-data-formatting): `pivot_wider()` and `pivot_longer()`

We will use two main packages in this session: `dplyr` and `tidyr`:

In [19]:
library(dplyr)
library(tidyr)

<br>

---

## Pipes

The pipe operator looks like this: `|>`. It takes the output of one function, and "pipes" it into the first argument of the next function.

But why would such a thing be useful? Well, here's some example code to hopefully demonstrate...

In [20]:
options(repr.plot.width=3.5, repr.plot.height=3, repr.matrix.max.rows=10)

In [21]:
data_1 <- starwars
data_2 <- filter(data_1, homeworld != "Alderaan")
data_3 <- mutate(data_2, height_inches = height/2.54)
data_4 <- select(data_3, name, height, height_inches)
data_5 <- arrange(data_4, height)

data_5

name,height,height_inches
<chr>,<int>,<dbl>
Ratts Tyerell,79,31.10236
Wicket Systri Warrick,88,34.64567
Dud Bolt,94,37.00787
R2-D2,96,37.79528
R5-D4,97,38.18898
...,...,...
Roos Tarpals,224,88.18898
Chewbacca,228,89.76378
Lama Su,229,90.15748


In [22]:
print(data_5)

# A tibble: 74 x 3
   name                  height height_inches
   <chr>                  <int>         <dbl>
 1 Ratts Tyerell             79          31.1
 2 Wicket Systri Warrick     88          34.6
 3 Dud Bolt                  94          37.0
 4 R2-D2                     96          37.8
 5 R5-D4                     97          38.2
 6 Sebulba                  112          44.1
 7 Gasgano                  122          48.0
 8 Watto                    137          53.9
 9 Mon Mothma               150          59.1
10 Cordé                    157          61.8
# i 64 more rows


What you can hopefully see is that we start with one dataframe, `starwars`. We then apply the `filter()`, `mutate()`, `select()`, and `arrange()` functions. Each time, we take the result of the last output, apply the function, and store the result in a new variable.

Rather than storing several variables that we are not interested in, another approach would be to nest the functions within each other's parentheses. For example:

In [23]:
# an example of nested data wrangling - difficult to read isn't it?
final_data <- arrange(
    select(
        mutate(
            filter(starwars, homeworld != "Alderaan"),
            height_inches = height/2.54
        ),
        name, height, height_inches
    ),
    height
)

This is really difficult to read, isn't it!?

What if we want code as readable as the first example, but without the unnecessary variables in between. Pipes are a perfect solution!

In [24]:
# a clear, readable example using pipes
final_data <- starwars |>
    filter(homeworld != "Alderaan") |>
    mutate(height_inches = height/2.54) |>
    select(name, height, height_inches) |>
    arrange(height)

Each line tells R what to do with the output of the last line. The output of the last line is always provided to the first input of the function on the next line.

<br>

### Check your Knowledge!

Rewrite the following snippets of code to use pipes (`|>`). You should check that the output matches the value in the last variable to be assigned in the non-piped example.

##### a)

In [25]:
filtered_naboo <- filter(starwars, homeworld=="Naboo")
naboo_characters <- pull(filtered_naboo, name)

##### b)

In [26]:
hws <- pull(starwars, homeworld)
unique_worlds <- sort(unique(hws))

##### c)

In [27]:
sw_filt <- filter(starwars, birth_year>50)
sw_grp <- group_by(sw_filt, species)
mass_summ <- summarise(sw_grp, M = mean(mass, na.rm=TRUE), SD = sd(mass, na.rm=TRUE))

##### d)

In [28]:
height_summ <- summarise(group_by(starwars, homeworld), mean_height=mean(height, na.rm=TRUE))

<br>

---

## Wide and Long Data Formatting

Imagine we have response-time data from a within-participant experimental design where participants all took part in two conditions ("A" and "B"). There are two sensible ways we can organise these data in a table.

##### Wide Format Example

* The two conditions are in separate columns (`A` and `B`)
* Each row contains data from one participant (`id`)

In [29]:
data.frame(
    id = c("s01", "s02", "s03", "s04"),
    A = c(240, 300, 264, 312),
    B = c(542, 366, 662, 430)
)

id,A,B
<chr>,<dbl>,<dbl>
s01,240,542
s02,300,366
s03,264,662
s04,312,430


##### Long Format Example

* One column now contains values from all conditions (`rt`)
* Another column tells us which condition each observation comes from (`condition`)
* Each participant now has two rows - one for each condition

In [30]:
data.frame(
    id = rep(c("s01", "s02", "s03", "s04"), each = 2),
    condition = rep(c("A", "B"), 4),
    rt = c(240, 542, 300, 366, 264, 662, 312, 430)
)

id,condition,rt
<chr>,<chr>,<dbl>
s01,A,240
s01,B,542
s02,A,300
s02,B,366
s03,A,264
s03,B,662
s04,A,312
s04,B,430


Wide and Long data formats are simply different ways of representing the data. It is often useful to switch between long and wide data formats. For example, long data format can make data visualisations easier with `ggplot2` (see [Data Visualisation](https://jackedtaylor.github.io/expra-wise24/introduction/data_vis) session), or can make it easier to calculate difference scores.

The `tidyr` package has two very handy functions for switching between wide and long data formats: [`pivot_wider()`](#pivot_wider) and [`pivot_longer()`](#pivot_longer).

<br>

### `pivot_wider()`

The `pivot_wider()` function lets us switch *from long to wide* data formats. As an example, let's start with a summary of heights in the `starwars` dataset, split by sex and species.

In [31]:
sex_summ <- starwars |>
    group_by(sex, species) |>
    summarise(avg_height = mean(height, na.rm=TRUE))

sex_summ

`summarise()` has grouped output by 'sex'. You can override using the `.groups`
argument.


sex,species,avg_height
<chr>,<chr>,<dbl>
female,Clawdite,168.00
female,Human,160.25
female,Kaminoan,213.00
female,Mirialan,168.00
female,Tholothian,184.00
...,...,...
male,Xexto,122.0000
male,Yoda's species,66.0000
male,Zabrak,173.0000


In [32]:
print(sex_summ)

# A tibble: 41 x 3
# Groups:   sex [5]
   sex            species    avg_height
   <chr>          <chr>           <dbl>
 1 female         Clawdite         168 
 2 female         Human            160.
 3 female         Kaminoan         213 
 4 female         Mirialan         168 
 5 female         Tholothian       184 
 6 female         Togruta          178 
 7 female         Twi'lek          178 
 8 hermaphroditic Hutt             175 
 9 male           Aleena            79 
10 male           Besalisk         198 
# i 31 more rows


We can reformat this to a wider format, with the average heights for each sex in separate columns. This will result in one species per row of the dataframe. There are two main parameters that we need to provide to `pivot_wider()`. These are `names_from` (where the new column names should come from) and `values_from` (where the new columns' values should come from).

In [33]:
pivot_wider(sex_summ, names_from=sex, values_from=avg_height)

species,female,hermaphroditic,male,none,NA
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Clawdite,168.00,NA,NA,NA,NA
Human,160.25,NA,182.3478,NA,NA
Kaminoan,213.00,NA,229.0000,NA,NA
Mirialan,168.00,NA,NA,NA,NA
Tholothian,184.00,NA,NA,NA,NA
...,...,...,...,...,...
Xexto,NA,NA,122,NA,NA
Yoda's species,NA,NA,66,NA,NA
Zabrak,NA,NA,173,NA,NA


In [34]:
pivot_wider(sex_summ, names_from=sex, values_from=avg_height) |> print()

# A tibble: 38 x 6
   species    female hermaphroditic  male  none  `NA`
   <chr>       <dbl>          <dbl> <dbl> <dbl> <dbl>
 1 Clawdite     168              NA   NA     NA    NA
 2 Human        160.             NA  182.    NA    NA
 3 Kaminoan     213              NA  229     NA    NA
 4 Mirialan     168              NA   NA     NA    NA
 5 Tholothian   184              NA   NA     NA    NA
 6 Togruta      178              NA   NA     NA    NA
 7 Twi'lek      178              NA  180     NA    NA
 8 Hutt          NA             175   NA     NA    NA
 9 Aleena        NA              NA   79     NA    NA
10 Besalisk      NA              NA  198     NA    NA
# i 28 more rows


As you can see, this results in several `NA`s. This highlights an important behaviour of `pivot_wider()` - missing combinations of `species` and `sex` are still included in the table, but the values are stored as `NA`, because there are no observations for these groups.

<br>

### Check your Knowledge!

Run the following code to generate the long-format data example from before. Then:

1. Use `pivot_wider()` to reformat the dataframe into the [wide format example shown earlier](#wide-format-example).
2. Use `mutate()` to calculate a difference score for each participant (as the response time in A minus the response time in B).

In [35]:
rts <- tibble(
    id = rep(c("s01", "s02", "s03", "s04"), each = 2),
    condition = rep(c("A", "B"), 4),
    rt = c(240, 542, 300, 366, 264, 662, 312, 430)
)

<br>

### `pivot_longer()`

The `pivot_longer()` function lets us switch *from wide to long* data formats. As an example, we can start with these made-up data of means and medians from three conditions.

In [36]:
# simulate two conditions' RTs - cell hidden but kept as may need changing at some point
N <- 1e6
B0 <- 1
Bb <- 0.5
Bc <- -0.5
e_sd <- 0.6
ndt <- 150

d <- tibble(cond = rep(c("A", "B", "C"), each=N)) |>
    mutate(
        b_dum = cond=="B",
        c_dum = cond=="C",
        y = ndt + exp(
            B0 + b_dum * Bb + c_dum * Bc +
                rnorm(n(), 0, e_sd)
        ) * 100
    )

summ <- d |>
    group_by(cond) |>
    summarise(mean_rt = mean(y), median_rt = median(y)) |>
    mutate(mean_rt = round(mean_rt, 2), median_rt = round(median_rt, 2))

summ

cond,mean_rt,median_rt
<chr>,<dbl>,<dbl>
A,475.63,422.13
B,687.09,598.86
C,347.40,314.78


In [37]:
rt_summ <- tibble(
    cond = c("A", "B", "C"),
    mean_rt = c(470.9, 686.30, 347.37),
    median_rt = c(422.32, 597.9, 314.89)
)

These data show two different estimators of the average response time - the mean and median. Let's imagine that we want all the averages in one column, `estimate`, while a new column (`type`) will tell us what kind of average we have. We can use `pivot_longer()` to do this.

The main two parameters of `pivot_longer()` are:
* `cols` - the columns to pivot into long format
* `names_to` - the name of the new column that should tell us what kind of average we have
* `values_to` - the name of the new column that should give us the actual values

In [38]:
pivot_longer(rt_summ, cols=c(mean_rt, median_rt), names_to="type", values_to="estimate")

cond,type,estimate
<chr>,<chr>,<dbl>
A,mean_rt,470.90
A,median_rt,422.32
B,mean_rt,686.30
B,median_rt,597.90
C,mean_rt,347.37
C,median_rt,314.89


In [39]:
pivot_longer(rt_summ, cols=c(mean_rt, median_rt), names_to="type", values_to="estimate")

cond,type,estimate
<chr>,<chr>,<dbl>
A,mean_rt,470.90
A,median_rt,422.32
B,mean_rt,686.30
B,median_rt,597.90
C,mean_rt,347.37
C,median_rt,314.89


<br>

### Check your Knowledge!

Starting with the dataframe below:
1. Use `pivot_longer()` to reformat the dataframe into the [long format example shown earlier](long-format-example).
2. Use `group_by()` and `summarise()` to calculate each participant's overall average (across both conditions).

In [40]:
rt_wide <- data.frame(
    id = c("s01", "s02", "s03", "s04"),
    A = c(240, 300, 264, 312),
    B = c(542, 366, 662, 430)
)

<br>

---

## Bonus: Discussion of Tidyverse

One of the advantages of using `tidyverse` packages like `dplyr` and `tidyr` is that they are arguably more readable. For instance, consider this example code using pipes from earlier in this chapter:

In [41]:
final_data <- starwars |>
    filter(homeworld != "Alderaan") |>
    mutate(height_inches = height/2.54) |>
    select(name, height, height_inches) |>
    arrange(height)

Here is how we would have to write this in base R:

In [42]:
final_data <- starwars
final_data <- final_data[final_data$homeworld != "Alderaan", ]
final_data$height_inches <- final_data$height / 2.54
final_data <- final_data[, c("name", "height", "height_inches")]
final_data <- final_data[order(final_data$height), ]

Despite the advantage in readability, the `tidyverse` approach is also often criticised for issues like ambiguity. For example, `tidyverse` code can be inherently ambiguous when read by a human. Should the values for the new column come from the variable we created called `height`, or from the existing column in the `starwars` dataframe, which is also called `height`?

In [43]:
height <- c(4, 4, 3, 1, 2, 6, 1, 3, 8, 9, 8, 1,
7, 2, 1, 9, 8, 1, 8, 3, 7, 3, 5, 6, 9, 8, 6, 1,
3, 2, 1, 9, 6, 8, 8, 1, 2, 9, 5, 4, 1, 3, 9, 3,
6, 1, 7, 4, 9, 8, 7, 5, 6, 3, 1, 5, 9, 3, 9, 4,
4, 7, 8, 6, 4, 5, 6, 3, 2, 9, 1, 7, 1, 6, 4, 6,
9, 6, 4, 5, 9, 5, 5, 4, 2, 3, 4)

new_data <- mutate(starwars, new_column = height)

As it happens, `dplyr` will check the current dataframe for variables first, and then check variables in the environment, so that the new column's values will be identical to `starwars$height`. The base R example from earlier doesn't suffer from this problem because whenever we refer to variables in the dataframe, we are explicit with `$` and `[]`.

If you interested in reading more, Norman Matloff has shared a well considered criticism of the `tidyverse` approach here: https://github.com/matloff/TidyverseSkeptic

You can always use personal preference when writing code. I would argue that `dplyr` and `tidyr` provide convenient and readable code for data wrangling. However, it's also important to recognise the shortcomings of the `tidyverse` approach. Learning base R, *as well* as the tidier approaches, can set you up with a much stronger understanding of R, and provide skills that transfer well to other programming languages that are becoming increasingly relevant in Psychology (e.g., `python`).